In [2]:
import pandas as pd

pop= pd.read_csv('state-population.csv')
areas= pd.read_csv('state-areas.csv')
abbrevs= pd.read_csv('state-abbrevs.csv')

In [5]:
print(pop.head()); print(areas.head()); print(abbrevs.head()) #5개 행만 출력

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [4]:
print(pop); print(areas); print(abbrevs)

     state/region     ages  year   population
0              AL  under18  2012    1117489.0
1              AL    total  2012    4817528.0
2              AL  under18  2010    1130966.0
3              AL    total  2010    4785570.0
4              AL  under18  2011    1125763.0
...           ...      ...   ...          ...
2539          USA    total  2010  309326295.0
2540          USA  under18  2011   73902222.0
2541          USA    total  2011  311582564.0
2542          USA  under18  2012   73708179.0
2543          USA    total  2012  313873685.0

[2544 rows x 4 columns]
                   state  area (sq. mi)
0                Alabama          52423
1                 Alaska         656425
2                Arizona         114006
3               Arkansas          53182
4             California         163707
5               Colorado         104100
6            Connecticut           5544
7               Delaware           1954
8                Florida          65758
9                Georgi

In [7]:
merged= pd.merge(pop, abbrevs, how= 'outer', left_on= 'state/region', right_on= 'abbreviation')
merged= merged.drop('abbreviation', 1) #중복되는 열 삭제

merged.head()

C:\Users\user\AppData\Local\Temp\ipykernel_3052\3649891298.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  merged= merged.drop('abbreviation', 1) #중복되는 열 삭제


,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [9]:
merged.isnull().any() #코드에 불일치하는 항목이 있는지 확인 -> 널 값을 가진 행 찾기, 일부 population, state 정보가 널 값

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [11]:
merged[merged['population'].isnull()].head() #population 값이 NaN인 데이터 찾기

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [13]:
#state 값이 널 값인 지역은 어디?
merged.loc[merged['state'].isnull(), 'state/region'].unique() #unique(): 데이터에 고유값들이 어떠한 종류들이 있는지 알고 싶을때 사용하는 함수
#state 열이 널값인 state/region 값

array(['PR', 'USA'], dtype=object)

In [15]:
#PR, USA라는 약어로 지정되어있었기 때문에 널 값을 가짐. 따라서 적절한 항목 채우기
merged.loc[merged['state/region']=='PR', 'state'] = 'Puerto Rico' #state/region=='PR'인 state 값은 Puerto Rico
merged.loc[merged['state/region']=='USA', 'state'] = 'United States'
merged.isnull().any()
#state 값은 널 값을 다른 값으로 채워넣음

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [17]:
#state 열에 더 이상 널 값이 없으니 연산 가능
final= pd.merge(merged, areas, on= 'state', how= 'left') #state 열을 keys 값으로, merged 항목을 기준으로 조인
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [18]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [19]:
#area 열에 있는 널 값 위치 확인
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [20]:
#전체 미국 면적을 담고 있지 않음 -> 적절한 값을 삽입하거나, 아예 삭제할 수 있음
#이 예제에서는 미국 전체의 인구 밀도가 관련이 없기 때문에 널 값 삭제
final.dropna(inplace= True) #결측치 삭제한 상태 바로 적용하기
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [21]:
#데이터 정리 완료
data2010= final.query("year==2010 & ages=='total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [22]:
#인구 밀도 계산하고 순서대로 표시
data2010.set_index('state', inplace= True) #'state' 열이 data2010의 인덱스
density= data2010['population'] / data2010['area (sq. mi)']

In [23]:
density.sort_values(ascending= False, inplace= True) #내림차순
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [24]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64